## **Import Libraries**

In [1]:
import numpy as np
import torch
from torch_geometric.nn import GCNConv
from torch_geometric.loader import DataLoader
import torch.nn.functional as F
from torch_geometric.data import Data

## **Load the Trained Model**

convert this class to a .py script later on!

In [2]:
class SimpleGCN(torch.nn.Module):
    def __init__(self, num_node_features, num_classes):
        super(SimpleGCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, 16)  # First GCN layer
        self.conv2 = GCNConv(16, num_classes)        # Second GCN layer

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x  # No log_softmax for regression

In [3]:
num_node_features = 4  # From your Data object, x=[713, 4]
num_classes = 1  # Assuming a regression task; adjust based on your specific task

model = SimpleGCN(num_node_features, num_classes)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)  # Using Adam optimizer

In [4]:
# paste specific model path
model_iteration = 'model_2024-02-15_22-14-57'

In [5]:
# Load the saved state_dict
model.load_state_dict(torch.load(f'pytorch_model/{model_iteration}.pth'))

<All keys matched successfully>

In [6]:
# Ensure you call model.eval() to set dropout and batch normalization layers to evaluation mode
model.eval()

SimpleGCN(
  (conv1): GCNConv(4, 16)
  (conv2): GCNConv(16, 1)
)

## **Load the Pytorch Data Object**

In [7]:
# Load your data object for prediction

new_data = torch.load('data/graph_data_predict.pt')
print('loaded')

loaded


In [8]:
# If 'new_data' is a single Data object, wrap it in a list for DataLoader compatibility
if isinstance(new_data, Data):
    new_data = [new_data]

In [9]:
# Create a DataLoader for your new dataset
data_loader = DataLoader(new_data, batch_size=1, shuffle=False)

## **Slicing Range**

In [10]:
# Assuming the first 592 out of 896 features are for sensors
sensor_indices = slice(0, 592)  # This creates a slice object for the first 592 features

## **Make Predictions**

In [11]:
# Make predictions
predictions = []
for data in data_loader:
    with torch.no_grad():
        output = model(data)
        sensor_output = output[sensor_indices]  # Adjusted slicing
        predictions.append(sensor_output.squeeze(-1).detach().numpy())
        
# 'predictions' now contains the predicted values for your new dataset

In [12]:
predictions

[array([428.22824, 430.43756, 432.6469 , 434.8563 , 437.06564, 439.275  ,
        441.48438, 443.69366, 445.903  , 448.11237, 450.32172, 452.53107,
        454.74042, 456.94977, 427.74857, 429.8106 , 431.87268, 433.93475,
        435.99677, 438.05887, 440.1209 , 442.18295, 444.24506, 446.30707,
        448.36914, 450.4312 , 452.49326, 454.55533, 456.6174 , 430.00433,
        432.06638, 434.12845, 436.1905 , 438.25256, 440.31464, 442.37668,
        444.43875, 446.5008 , 448.56287, 450.62494, 452.68695, 454.74902,
        456.8111 , 458.87314, 430.67303, 432.73514, 434.79718, 436.85922,
        438.9213 , 440.98337, 443.0454 , 445.10748, 447.16953, 449.23157,
        451.29364, 453.3557 , 455.4178 , 457.47983, 459.54187, 432.3059 ,
        434.36798, 436.43   , 438.4921 , 440.55417, 442.6162 , 444.67828,
        446.74033, 448.80237, 450.86444, 452.9265 , 454.98856, 457.05063,
        459.1127 , 461.1747 , 433.48026, 435.5423 , 437.6044 , 439.66644,
        441.72852, 443.7906 , 445.8526

In [13]:
# Display the shape of predictions
predictions_array = np.array(predictions)
print("Shape of predictions array:", predictions_array.shape)

Shape of predictions array: (1, 592)


In [14]:
print("Output shape:", output.shape)
print("Sensor output shape:", sensor_output.shape)

Output shape: torch.Size([896, 1])
Sensor output shape: torch.Size([592, 1])


So, in summary, the shape (1, 896, 1) suggests that you have one sample, each containing 896 elements, and each of these elements is a single output value.

## **Export to csv**

In [15]:
predictions_array = np.vstack(predictions)
# Reshape predictions_array to have shape (592, 1)
predictions_array = predictions_array.T
print("Shape of sensor predictions array:", predictions_array.shape)

Shape of sensor predictions array: (592, 1)


In [16]:
# Export rounded predictions to CSV in the same folder as the script
np.savetxt("sensor_predictions.csv", predictions_array, delimiter=",")
print('saved')

saved


# note theres 2 options to deal with the masking / indexing issue
1. save the sensor_mask as a numpy array in the training script and retrive it here
2. manually dictate the indices to slice at

i took option 2 cos easier in grasshopper component later on